# Wk18 Lecture02 CodeAlong: UFOs

## Learning Objectives

- By the end of this CodeAlong, students will be able to:
   - Calculate time series statistics (rolling mean/std/diff/pct_change
   - Perform feature engineering for time series EDA 
   - Aggregate time series using date parts to answer stakeholder questions.

    

# 🕹️Part 1) Preparing Irregular-Interval Time Series

### Overview from Last Lecture

- 1) [ ] Convert the dates & times to a single column (if needed).
- 2) [ ] Convert the datetime column  (most likely a string) to a datetime data type.
- 3) [ ] Set the datetime column as the Series/DataFrame index
- 4) [ ] Resample the time series to the desired/correct frequency using the desired/correct aggregation method.
- 5) [ ] Impute null values (if required)


### UFO Sightings

- UFO Sightings: https://www.kaggle.com/datasets/NUFORC/ufo-sightings 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticks
import seaborn as sns


import missingno as miss
import datetime as dt
import statsmodels.tsa.api as tsa

plt.rcParams['figure.figsize'] = [10,5]

In [ ]:
ufo  = pd.read_csv("Data/ufos-kaggle/scrubbed.csv", low_memory=False)
ufo

In [ ]:
ufo.info()

In [ ]:
## convert latitude to float
ufo['latitude'] = ufo['latitude'].astype('float64')
ufo.info()

In [ ]:
## Try again to convert type
ufo['latitude'] = ufo['latitude'].str.replace('q','')

ufo['latitude'] = ufo['latitude'].astype('float64')
ufo.info()

## 1) Convert the dates & times to a single column (if needed).

Datetime is already one column.  Nothing to do here.

## 2) Converting Date Cols to Datetime

In [ ]:
## convert datetime to datetime
ufo['datetime'] = pd.to_datetime(ufo['datetime'])

#### Handling Errors with pd.to_datetime

- Can use the `errors` argument for pd.to_datetime:
    - "raise" (default): raise an exception when errors happen
    - 'ignore': ignores the errors and returns the original value for that row. 
        - NOT RECOMMENDED: the entire column will not be datetime.
    - 'coerce': convert any bad datetime values to null values (NaT - NotATime)

>- **Branch point: we have a choice on how we deal with the bad timestamps.**
    -  Do we coerce them, make then null values, and drop them? Potentially losing a lot of data.
    - Or do we investigate a bit more to see if we can fix the problem without losing data.
    
    
- Let's see how much data we would lose if we chose to coerce the bad values:

In [ ]:
## saving a copy of original datetime column as datetime-original
ufo['datetime-original'] = ufo['datetime'].copy()
ufo

In [ ]:
##check nulls before coercing errors
ufo.isna().sum()

In [ ]:
## Create a datetime-coerce column using pd.to_Datetime with errors = "coerce"
ufo['datetime-coerce'] = pd.to_datetime(ufo['datetime-original'], errors='coerce')
ufo

In [ ]:
## How many null values did we create?
ufo.isna().sum()

In [ ]:
## What % of dates became null?
ufo.isna().sum() / len(ufo) *100

#### Acceptable # of Rows Lost, but there is another solution!

In [ ]:
ufo

In [ ]:
## inspecting just the bad rows
bad_rows = ufo[ufo['datetime-coerce'].isna()]
bad_rows

In [ ]:
## let's try to convert the bad rows again
try:
    pd.to_datetime(bad_rows['datetime-original'])
except Exception as e:
    display(e)

> 🤔"`ParserError: hour must be in 0..23: 10/11/2006 24:00 present at position 0`"

In [ ]:
bad_rows['datetime-original']

> Panda's is confused by 24:00. It doesn't know if we mean 0:00 of the NEXT day or if we mean the 11:59 pm (23:59) the same day

In [ ]:
## replace 24:00 with 23:59
ufo['datetime-fixed'] = ufo["datetime-original"].str.replace("24:00", "23:59")
ufo

In [ ]:
## use the fixed-datetime column to make the datetime col
ufo['datetime'] = pd.to_datetime(ufo["datetime-fixed"])
ufo

In [ ]:
ufo.isna().sum()

## 3) Setting datetime index

In [ ]:
## Create ufo_ts by setting the datetime index
ufo_ts = ufo.set_index('datetime')
ufo_ts

In [ ]:
## Drop any remaining datetime columns
dt_cols = [c for c in ufo_ts.columns if 'datetime' in c]
dt_cols

In [ ]:
# check the index to confirm its datetime
ufo_ts = ufo_ts.drop(columns=dt_cols)
ufo_ts

In [ ]:
ufo_ts.index

## 4) Resample Data to Desired Frequency

What frequency should we resample our data to?  This requires some thinking

### Let's visualize Our Data

In [ ]:
## Plot the full dataset
# ufo_ts.plot()

> Hmmmm.... what are we *trying* to visualize?



**What do we really want to know about UFO's?**
- Duration of sighting?
- Location of sighting?
- Number of sightings?

In [ ]:
ufo_ts

### Converting to Daily Frequency

**We want to quantify the number of events that occurred within each interval.**

>- Q: How could we do this? (there's several ways)

1. Add a column of all 1s and sum them up for each day
2. Resample by day and aggregate by the number of entries for each day


In [ ]:
## A.1) Add a count column with a value of 1 for each row and then use reasmple().sum()
demo_ts = ufo_ts.copy()
demo_ts['Count'] = 1
demo_ts

In [ ]:
# A.2) resample and take sum
demo_ts.resample("D").sum()

In [ ]:
## Option B) use .size
ufo_ts.resample("D").size()

### Make `ts` from ufo_ts

In [ ]:
## Resample as daily data 
ts = ufo_ts.resample('D').size()
ts

In [ ]:
# plot the ts
ts.plot();


Is all of this data relevant and interesting?  When did sightings really start becoming significant?


In [ ]:
## keep only 1950 and later
ts = ts.loc["1950":]
ts.plot()

# Part 2) Aggregating Full Dataset Using Date Parts

## 📝 **Stakeholder Questions to Answer**

**ANSWER TOGETHER:**
- 1) What Month and Year had the most sightings? (and how many sightings were there?)

- 2) Which month of the year has had the highest total number of reported sightings??
- 3) Is there a seasonal pattern to UFO sightings? If so, how long is the season?

- 4) Which US holiday has the largest number of sightings?
___
**ANSWER SELECTED Q's IN BREAKOUT ROOMS**

- 5) Which year had the highest % increase in sightings compared to previous years? (since 1950)

- 6) What day of the week has the highest reported sightings?

- 7) At what time of day (hour) do most sightings occur?

- 8) Which US state has the most sightings?

- 9) Which country had the largest proportion of sightings for the year 2000?

- 10) Have the types/shapes of UFO's witness changed over time?
    - Tip: use only the 4 most common shapes

### Making `eda_df` for answering questions

In order to access the datetime library of methods in Pandas, we will need to convert `datetime` back into a column.  A datetime index does not have the methods we need.

In [ ]:
## making eda_df with date as a column instead of index
eda_df = ufo_ts.reset_index()
eda_df

## Feature Engineering: Date Parts

- Datetime objects have:
    - year
    - month
    - month_name()
    - day
    - day_name()
    - hour
    - seconds
    
- Pandas has a `.dt.` accessor to use datetime methods on an entire column at once.

In [ ]:
## feature engineering for dates
eda_df['year'] = eda_df['datetime'].dt.year
eda_df['month'] = eda_df['datetime'].dt.month_name()
eda_df['day of month'] = eda_df['datetime'].dt.day
eda_df['day of week'] = eda_df['datetime'].dt.day_name()
eda_df['hour'] = eda_df['datetime'].dt.hour
eda_df

> Let's add a "weekend" feature that will be True if the day was a Saturday or Sunday.

In [ ]:
## let's add a weekend feature
eda_df['weekend'] = eda_df['day of week'].isin(['Saturday','Sunday'])
eda_df

#### Let's add a column for the decade

In [ ]:
## Calculate decade by subtracting the remainder and div by 10
eda_df['decade'] = eda_df['year'] - eda_df['year']%10
eda_df

## 🕹️ Answering Stakeholder Questions (Together)

### Making `eda_ts` & `ts`

In [ ]:
## Making eda_ts with dt index
eda_ts = eda_df.set_index('datetime')
eda_ts

### 1) What Month/Year had the most sightings? (and how many sightings were there?)


In [ ]:
## make a ts that is resampled to monthly
ts = eda_ts.resample("M").size()
ts

In [ ]:
## get the date of the max sightings


In [ ]:
# how many sightings?


In [ ]:
## Plot the ts and add vertical line at month with most sightings


### 2) Which month of the year has had the highest total number of reported sightings?


### 3) Is there a seasonal pattern to UFO sightings? If so, how long is the season?

### Seasonality

In [ ]:
import statsmodels.tsa.api as tsa

In [ ]:
## Isolate just years since 2000 to capture recent trends


In [ ]:
## plot the sliced ts


In [ ]:
## Isolate trend and seasonal components with seasonal_decompose()


In [ ]:
## Plot the decomposition


In [ ]:
## separate seasonal component and plot


### 4) Which US holiday has the largest number of sightings?

#### Feature Engineering: Holidays

In [ ]:
# !pip install holidays
import holidays
import datetime as dt
from holidays import country_holidays

In [ ]:
## Create an instance of the US country holidays.


In [ ]:
## create a test holiday 
test = "01/01/2015"
test

In [ ]:
## use .get() to test the api 


In [ ]:
## Map the api's .get method onto the df to get all holidays

## Check the unique holidays


Apparently Juneteenth has made it on there yet.

#### Answer to which holiday has most sightings:

#### Wait...when did **that** movie come out?

In [ ]:
release_date= '1997-07-03'

In [ ]:
## Plot the # of sightings over time and annotate the release date


### 7-day Rolling Mean of Daily Sightings
This is so noisy, let's try plotting a rolling mean.

## 🏓**Breakout Rooms: Answering Stakeholder Questions**

**Choose 1-2 of the remaining questions and work in breakout rooms to answer them:**
- 5) Which year had the highest % increase in sightings compared to previous years?
- 6) What day of the week has the highest reported sightings?
- 7) At what time of day (hour) do most sightings occur?
- 8) Which US state has the most sightings?
- 9) Which country had the largest proportion of sightings for the year 2000?
- 10) Have the types/shapes of UFO's witness changed over time?
    - Tip: use only the 4 most common shapes



### 5) Which year had the highest % increase in sightings compared to previous years? (since 1950)

### 6) What day of the week has the highest reported sightings?

### 6) Which country had the largest proportion of sightings for the year 2000?

### 7) Have the types/shapes of UFO's witness changed over time?

___
# Bonus: Plotly Express

In [ ]:
import plotly.express as px
import plotly.io as pio

### Map Over Time

In [ ]:
eda_df = eda_df.sort_values('decade')
eda_df.columns = eda_df.columns.str.strip()
eda_df['latitude'] = pd.to_numeric(eda_df['latitude'], errors='coerce')
eda_df.head()

In [ ]:
px.scatter_geo(data_frame=eda_df, lat='latitude',lon='longitude', animation_frame="decade",
              template='ggplot2')